In [189]:
import pytesseract
from PIL import Image
from PIL import ImageFile
import pandas as pd
import re
import numpy as np
import warnings
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from hanspell import spell_checker

ImageFile.LOAD_TRUNCATED_IMAGES = True
pytesseract.pytesseract.tesseract_cmd = R'C:\Program Files\Tesseract-OCR\tesseract'
warnings.filterwarnings('ignore')

## 기존 정보 및 사전 작업

### 기존 데이터

In [2]:
df=pd.read_csv('./data/kurly_classify_allergy_gluten.csv',encoding='utf-8')
df

,category,brand,item,allergy,ingredient,classify,gluten
0,국/반찬/메인 요리,풀무원,국산콩 나또 3종,"국산콩생나또,대두,밀,쇠고기,검은콩생나또,대두,밀,쇠고기,국산콩와사비나또,대두,밀,...","고등어,아황산류,메밀,옥수수유옥수수,복숭아,오징어,홍합포함,설탕,조개류,전복,대두,...",general,발견사항 없음
1,국/반찬/메인 요리,kurly,전통 시장 물떡 (2팩),"밀,대두,우유,계란,우유,메밀,땅콩,고등어,게,새우,돼지고기,복숭아,토마토,아황산류...","조개류,정제소금,주정,떡류,조기,소스소스,참치,비타민,가공소금,오징어,구연산,복합조...",pesco,발견사항 없음
2,국/반찬/메인 요리,모두의맛집,소문난 원조 조방낙지 낙곱새,"대두,밀,새우,쇠고기,조개류(굴),난류,우유,메밀,땅콩,고등어,게,돼지고기,복숭아,...","마늘,조개류,전복,내두유,콩기름,홍합포함",pesco,발견사항 없음
3,국/반찬/메인 요리,치즈룸x테이스팅룸,전복 솥밥 리조또,"전복미역밥,우유,조개류(전복),전복내장크림소스,대두,밀,우유,계란,닭고기,조개류(굴...","마늘,고등어,호두,유청,소고기,껏일,포도당,구연산,가공이크림,토코페롤,변성전분,주정...",general,발견사항 없음
4,국/반찬/메인 요리,피코크,금돼지식당 통삽겹김치찌개,"대두,밀,돼지고기,쇠고기,알류,우유,메밀,땅콩,고등어,게,새우,복숭아,토마토,아황산...","마늘,고등어,향미증진제,콜산,호두,복숭아,홍합포함,인산이전분,닭고기,고춧가루,진한양...",general,발견사항 없음
...,...,...,...,...,...,...,...
2925,간식/과자/떡,라라스윗,발렌타인 기프트 세트,"우유,밀,계란,대두,땅콩,호두,메밀,복숭아,토마토,돼지고기혼입가능","알룰로스,당류",vegan,발견사항 없음
2926,간식/과자/떡,발스낵,그랜드 감자칩 2묶음 6종 (택1),"씨솔트,와사비,칠리앤드라임,사우어크림앤드어니언,밀,우유,샐러리,겨자,사워크림앤허브,...","글루,정제소금,오일,고추냉이,정제수정제소금,감자플레이크,리브뉴클레오티드이나트롬,감자...",vegan,발견사항 없음
2927,간식/과자/떡,리터 스포트,미니 초콜릿믹스 9P,없음,"준초콜릿,옥수수",vegan,발견사항 없음
2928,간식/과자/떡,청우,아홉 번 구워만든 죽염캔디 38g,"땅콩,대두,우유,복숭아","무엇,자일리톨,박하유",vegan,발견사항 없음


### 분류 기준표

In [3]:
with open('./list/meat_f.txt','r') as file:
    lst=file.readlines()

meat_lst=[]
for word in lst:
    word=word.replace('\n','')
    word=word.replace(' ','')
    if len(word)>0:
        meat_lst.append(word)

In [5]:
with open('./list/sea_f.txt','r') as file:
    lst=file.readlines()

sea_lst=[]
for word in lst:
    word=word.replace('\n','')
    word=word.replace(' ','')
    if len(word)>0:
        sea_lst.append(word)

In [6]:
with open('./list/milk_f.txt','r') as file:
    lst=file.readlines()

milk_lst=[]
for word in lst:
    word=word.replace('\n','')
    word=word.replace(' ','')
    if len(word)>0:
        milk_lst.append(word)

In [7]:
with open('./list/egg_f.txt','r') as file:
    lst=file.readlines()

egg_lst=[]
for word in lst:
    word=word.replace('\n','')
    word=word.replace(' ','')
    if len(word)>0:
        egg_lst.append(word)

In [8]:
with open('./list/honey_f.txt','r') as file:
    lst=file.readlines()

honey_lst=[]
for word in lst:
    word=word.replace('\n','')
    word=word.replace(' ','')
    if len(word)>0:
        honey_lst.append(word)

### 추천시스템 함수

In [176]:
def find_sim_ingredient(df,sorted_ind,product_name,top_n=10,cate=0,vegan=0):
    
    title_product=df[df['item']==product_name]
    title_index=title_product.index.values

    similar_indexes=sorted_ind[title_index]
    similar_indexes=similar_indexes[similar_indexes!=title_index]

    similar_indexes=similar_indexes.reshape(-1)
    option_lst=similar_indexes

    # 카테고리 옵션
    if cate==1:
        option_lst=[]
        category=df.loc[df['item']==product_name,'category'].values
        cate_indexes=df.loc[df['category']==category[0]].index.values
    
    # 비건단계 옵션1 -같은 종류의 단계만 추천
    if vegan==1:
        option_lst=[]
        vegan_level=df.loc[df['item']==product_name,'classify'].values
        same_vegan_indexes=df.loc[df['classify']==vegan_level[0]].index.values

    # 비건단계 옵션2 -비건단계당 2개씩 추천
    elif vegan==2:
        option_lst=[]
        vegan_check={'pesco':0,'lacto_ovo':0,'lacto':0,'ovo':0,'vegan':0,'honey':0}
        cnt=0
            
    # 비건단계 옵션3 - 비건 제품만 추천
    elif vegan==3:
        option_lst=[]
        vegan_indexes=df.loc[df['classify']=='vegan'].index.values

    # 옵션 적용
    if cate==1 and vegan==0: # 카테고리
        for idx in similar_indexes:
            if idx in cate_indexes:
                option_lst.append(idx)
    
    elif cate==0 and vegan==1: # 같은비건단계
        for idx in similar_indexes:
            if idx in same_vegan_indexes:
                option_lst.append(idx)

    elif cate==0 and vegan==2: # 비건단계별
        for idx in similar_indexes:
            vegan_level=df.iloc[idx,-3]
            if vegan_level!='general' and vegan_check[vegan_level]<2:
                vegan_check[vegan_level]+=1
                option_lst.append(idx)
                cnt+=1
            elif cnt==12:
                break
            else:
                continue
    
    elif cate==0 and vegan==3: # 비건제품만
        for idx in similar_indexes:
            if idx in vegan_indexes:
                option_lst.append(idx)

    elif cate==1 and vegan==1: # 카테고리, 같은비건단계
        for idx in similar_indexes:
            if idx in cate_indexes and idx in same_vegan_indexes:
                option_lst.append(idx)

    elif cate==1 and vegan==2: # 카테고리, 비건단계별
        for idx in similar_indexes:
            if idx in cate_indexes:
                vegan_level=df.iloc[idx,-3]
                if vegan_level!='general' and vegan_check[vegan_level]<2:
                    vegan_check[vegan_level]+=1
                    option_lst.append(idx)
                    cnt+=1
                elif cnt==12:
                    break
                else:
                    continue

    elif cate==1 and vegan==3:  # 카테고리, 비건제품만
        for idx in similar_indexes:
            if idx in cate_indexes and idx in vegan_indexes:
                option_lst.append(idx)

    return df.iloc[option_lst][:top_n]

In [12]:
df['ingredient']=df['ingredient'].apply(lambda x:x.replace(',',' '))

### 글루텐 정보

In [13]:
glu=['글루테닌','글리아딘','보리','귀리','밀','강력분','중력분','박력분','밀가루']

### 불용어

In [49]:
stopwords=['미국산','외국산','인도','호주','브라질','말레이시아산','함유','함유로']

## 새로운 데이터 입력

### 1) 식품 정보 입력 및 성분 사진 인식

In [57]:
print('식품 정보를 입력하세요')
brand=[] ; item=[]; category=[]

brand.append(input(print('1) 식품 브랜드:')))
print('=>',brand[0])

item.append(input(print('2) 식품 이름:')))
print('=>',item[0])

category.append(input(print('3) 식품 카테고리:(국·반찬·메인요리/샐러드·간편식/면·양념·오일/생수·음료·우유·커피/간식·과자·떡/베이커리·치즈·델리)')))

print('=>',category[0])

category[0]=category[0].replace('·','/')

식품 정보를 입력하세요
1) 식품 브랜드:
=> 오리온
2) 식품 이름:
=> 고래밥
3) 식품 카테고리:(국·반찬·메인요리/샐러드·간편식/면·양념·오일/생수·음료·우유·커피/간식·과자·떡/베이커리·치즈·델리)
=> 간식·과자·떡


In [37]:
print('식품 성분 사진을 불러옵니다.')
img=Image.open('./img/image1_ingre.jpg')
result=pytesseract.image_to_string(img,lang='kor')

식품 성분 사진을 불러옵니다.


In [38]:
result

'밀가루(밀 미국산), 알파옥수수분말[옥수수 외국산(인도, 호주, 브라질)], 변성전\n분 , 식물성유지1(팔유 말레이시아산, 폴레인유 말레이시아산), 볶음양넘맛씨즈닝 4.3 %\n[감미료 아스파탐(페닐알라닌 함유)], 식물성유지2, 갈색설탕, 백설탕, 유단백혼합분말, 전\n분 4\n\n, 바베큐양념페이스트, 혼합치스분말, 패각칼슴, 식염, 산도조절제, 유화제\n밀, 달갈, 우유, 대두, 토마토, 돼지고기, 쇠고기, 오징어, 조개류(굴) 함유\n\n로 2 7 19919\n\n'

In [190]:
result=re.sub('[()\[\]]',',',result)
result=result.replace('\n','')
result=re.sub('[^가-힣,]',',',result)
result=re.sub(',+',',',result)
result

'밀가루,밀,미국산,알파옥수수분말,옥수수,외국산,인도,호주,브라질,변성전분,식물성유지,팔유,말레이시아산,폴레인유,말레이시아산,볶음양넘맛씨즈닝,감미료,아스파탐,페닐알라닌,함유,식물성유지,갈색설탕,백설탕,유단백혼합분말,전분,바베큐양념페이스트,혼합치스분말,패각칼슴,식염,산도조절제,유화제밀,달갈,우유,대두,토마토,돼지고기,쇠고기,오징어,조개류,굴,함유로,'

In [200]:
ingredient=''
for word in result.split(','):
    if word in stopwords:
        continue
    else:
        ingredient=ingredient+','+word

In [150]:
input_data=pd.DataFrame({'brand':brand,'item':item,'ingredient':ingredient,'category':category})
input_data

,brand,item,ingredient,category
0,오리온,고래밥,",밀가루,밀,알파옥수수분말,옥수수,변성전분,식물성유지,팔유,폴레인유,볶음양넘맛씨즈닝...",간식/과자/떡


### 2) 비건단계 식품 분류

In [151]:
input_data['classify']='0'
flag=0
milk=0
egg=0
for word in input_data['ingredient'][0].split(','):
    if word in meat_lst:
        flag=1
        break
if flag==0:
    for word in input_data['ingredient'][0].split(','):
        if word in sea_lst:
            flag=2
            break
if flag==0:
    for word in input_data['ingredient'][0].split(','):
        if word in milk_lst:
            flag=3
            milk+=1
            break
    for word in input_data['ingredient'][0].split(','):
        if word in egg_lst:
            flag=3
            egg+=1
            break
if flag==0:
    for word in input_data['ingredient'][0].split(','):
        if word in honey_lst:
            flag==4
            break
            
if flag==1:
    classify='general'
elif flag==2:
    classify='pesco'
elif flag==3:
    if milk>0 and egg==0:
        classify='lacto'
    elif milk==0 and egg>0:
        classify='ovo'
    else:
        classify='lacto ovo'
elif flag==4:
    classify='honey'
else:
    classify='vegan'
input_data['classify'][0]=classify
print('['+input_data['item'][0]+']은 '+input_data['classify'][0]+'식품입니다.')

[고래밥]은 general식품입니다.


### 3) 알레르기 확인

In [75]:
user_allergy=[]
user_input=''
print('알레르기 정보를 입력하세요.\n다 입력하신 후 \"끝\"을 입력해주세요.')
while user_input!='끝':
    user_input=input()
    print(user_input)
    user_allergy.append(user_input)
user_allergy=user_allergy[:-1]

알레르기 정보를 입력하세요.
다 입력하신 후 "끝"을 입력해주세요.
땅콩
복숭아
끝


In [183]:
allergy_lst=[]
for word in input_data['ingredient'][0].split(','):
    if word in user_allergy:
        allergy_lst.append(word)
if len(allergy_lst)>0:
    print('['+input_data['item'][0]+']에는 고객님이 입력하신 알레르기 중 '+str(allergy_lst)+'가 존재합니다. 조심하세요!')
    input_data['allergy']=','.join(allergy_lst)
else:
    print('['+input_data['item'][0]+']에는 고객님이 입력하신 알레르기가 없어요. 안심하세요!')
    input_data['allergy']='발견사항 없음'

[고래밥]에는 고객님이 입력하신 알레르기가 없어요. 안심하세요!


### 4) 글루텐 확인

In [185]:
glu_lst=[]
for word in input_data['ingredient'][0].split(' '):
    if word in glu:
        glu_lst.append(word)
if len(glu_lst)>0:
    print('['+input_data['item'][0]+']에는 글루텐이 포함되어 있습니다.')
    input_data['gluten']=','.join(glu_lst)
else:
    print('['+input_data['item'][0]+']에는 글루텐이 포함되어 있지 않습니다.')
    input_data['gluten']='발견사항 없음'

[고래밥]에는 글루텐이 포함되어 있습니다.


### 5) 추천시스템

In [152]:
input_data['ingredient']=input_data['ingredient'].apply(lambda x:x.replace(',',' '))

In [83]:
# 옵션 입력
print('카테고리 옵션을 선택하세요')
print('1) 같은 카테고리 식품만 궁금해요')
print('2) 다른 카테고리 식품도 궁금해요')
cate_option=input('1 / 2')

print('=> '+cate_option+'번 옵션 선택')

if cate_option=='1':
    cate_option=1
else:
    cate_option=0

print('\n비건 옵션을 선택하세요')
print('1) 비건 단계 상관없이 모두 궁금해요')
print('2) 같은 비건 단계 식품만 궁금해요')
print('3) 다른 비건단계 식품들도 궁금해요')
print('4) 비건인 식품만 궁금해요')
vegan_option=input('1 / 2 / 3 / 4')

print('=> '+vegan_option+'번 옵션 선택')

if vegan_option=='1':
    vegan_option=0
elif vegan_option=='2':
    vegan_option=1
elif vegan_option=='3':
    vegan_option=2
else:
    vegan_option=3

카테고리 옵션을 선택하세요
1) 같은 카테고리 식품만 궁금해요
2) 다른 카테고리 식품도 궁금해요
=> 1번 옵션 선택

비건 옵션을 선택하세요
1) 비건 단계 상관없이 모두 궁금해요
2) 같은 비건 단계 식품만 궁금해요
3) 다른 비건단계 식품들도 궁금해요
4) 비건인 식품만 궁금해요
=> 3번 옵션 선택


In [187]:
recom_df=pd.concat([input_data,df]).reset_index(drop=True)
count_vect=CountVectorizer(min_df=0,ngram_range=(1,2))
text_mat=count_vect.fit_transform(recom_df['ingredient'])
text_sim=cosine_similarity(text_mat,text_mat)
text_sim_sorted_ind=text_sim.argsort()[:,::-1]

print('입력한 상품과 비슷한 제품을 보여드려요')
find_sim_ingredient(recom_df,text_sim_sorted_ind,input_data['item'].values[0],10,cate_option,vegan_option)

입력한 상품과 비슷한 제품을 보여드려요


,brand,item,ingredient,category,classify,allergy,gluten
1875,오리온,예감 치즈 그라탕 204g,폴란드산 산도조절제 식물성유지,간식/과자/떡,lacto,"밀,우유,대두,쇠고기,오징어,새우",발견사항 없음
1570,칠갑농산,우리쌀 떡국떡 800g,우유 대두 계란,간식/과자/떡,lacto_ovo,"난류(계란),우유,대두,밀",발견사항 없음
1593,오리온,고래밥 20g x 10입,백설탕 치즈분말 산도조절제 유화제 전분 갈색설탕 아스파탐 분말,간식/과자/떡,lacto,"밀,달걀,우유,대두,토마토,돼지고기,쇠고기,오징어,조개류(굴)",발견사항 없음
2901,Gnaw,드라큐바 100g,전분,간식/과자/떡,vegan,"우유,대두,밀,땅콩,혼입가능성있음",발견사항 없음
2915,푸드도슨트,페스츄리 한입약과 160g,대두,간식/과자/떡,vegan,"밀,대두",발견사항 없음
933,서울마님,찹쌀옹심이,우유 대두 아황산류 새우 메밀땅콩,간식/과자/떡,pesco,"알류,우유,메밀,땅콩,대두,밀,고등어,게,새우,돼지고기,복숭아,토마토,아황산류,호두...",발견사항 없음
1034,농심,자갈치 300g (지퍼백),우유 산도조절제 전복 과자 대두 땅콩 토마토 새우 미쇠고기 소맥분 당류 홍합포함,간식/과자/떡,pesco,"새우,대두,우유,밀,쇠고기,게,토마토,돼지고기,오징어,닭고기,계란,땅콩,잣,조개류(...",발견사항 없음
1677,해마루,파주 장단콩 팥소빵 (5개입),설탕 밀가루 산도조절제 산도조절제전분 변성전분 앗술 베이킹파우더 대두 양조간장 방류...,간식/과자/떡,ovo,"계란,밀,대두",밀가루
1730,마요라,로마 마리에 골드 초코 비스킷 2개입 번들,밀가루 설탕 정제소금 시럽 버터 코코아 식물성유지 탄산수소나트롬 팽창제 탄산수소암모...,간식/과자/떡,lacto_ovo,"밀,우유,계란,대두,견과류",밀가루
1523,오리온,초코파이 바나나맛 12입,변성전분 때지 전란액 식물성크림 전분 혼합제제 합성향료 당류,간식/과자/떡,ovo,"밀,달걀,우유,대두,돼지고기,쇠고기,땅콩,복숭아,게,조개류",발견사항 없음
